In [1]:
# import libraries
#import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import gzip
import pickle
import os
from urllib.request import urlretrieve
import numpy as np
from matplotlib import image


def conv(image, params, s = 1, pad = 1 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_depth = f.shape[1] #f1 = 1 , f2 = 8
    f_size = f.shape[2] 
        

    h_range = int((image.shape[1] - f_size) / s) + 1     # (W - F + 2P) / S  
    w_range = int((image.shape[2] - f_size) / s) + 1     # (W - F + 2P) / S  
    np_o = np.zeros((f_num, h_range, w_range))
    for z in range(f_num): # Number of filters
        for _h in range(h_range):      
            for _w in range(w_range):
                np_o[z, _h, _w] = np.sum(np.multiply(image[:, _h*s : _h*s + f_size, _w*s : _w*s + f_size] , f[z, :, :, :])) + b[z]
    
    return np_o
def conv_alg(image, params, s = 1, pad = 0 ): # s = 1 (conv stride)
    #f1 shape : (num_filters,input channels, f_h, f_w)
    #image shape: (channels, height, width)
    [f, b] = params
    if(pad >= 1):
        tmp = np.zeros((image.shape[0],image.shape[1]+2*pad ,image.shape[2]+2*pad))
        tmp[:,pad:-pad,pad:-pad] = image
        image = tmp    
        
    f_num = f.shape[0]
    f_size = f.shape[2] 
    
    np_o_t = np.zeros((f_num, image.shape[1] + (f_size-1), image.shape[2]+ (f_size-1)))
    print(np_o_t.shape)
    for z in range(f_num): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                np_o_t[z,_h:(_h+f_size),_w:(_w+f_size)] += np.sum((image[:, _h, _w].reshape(-1,1,1)*f[z, :, :, :]),axis=0)
        np_o_t[z,:,:] += b[z]*np.ones((image.shape[1] + (f_size-1),image.shape[2] + (f_size-1)))
    return np_o_t[:,1:-1,1:-1]#(f_size-1):-(f_size-1), (f_size-1):-(f_size-1)]

def convTransp(image, params, s = 1, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f = np.rot90(f, 2, (2,3))
    params = [f, b]
    input_s = image.shape[1]
    target_dim = input_s*2 #final dim, after conv
    required_pad = 1 #always for filter 2x2 ,stride 1, zero insertion 1 and main target to double dim
    #make our new custom input
    size = input_s*2 +1
    new_in = np.zeros((image.shape[0], size, size))
    
    ### OR just: np.pad(image[:,:,:],2,'constant') # Important, we must loop with respect to the 1st dim
    for i in range(1, target_dim, 2):
        for j in range(1, target_dim, 2):
                new_in[:, i, j] = image[:, i//2, j//2]
    #now we do a normal convolution(pad = 0)
    res = conv(new_in, params, 1, 0) #thats the final result with target_dim
    return res, new_in # we will need new_in so we wont calc it again for the backprop
def convTransp_alg(image, params, s = 2, pad = 1):
    #s is always 1, upsample kernel = 2
    #zero insertions between pixels s_downsampled -1 = 2-1 = 1
    #required padding in order to double my dimensions with the given data:
    #(i-1)*2 + k -2p = output size, where our padding is k - p -1 = 2-0-1=1(we assume p=0)
    [f, b]=params
    f_size = f.shape[2]
    res = np.zeros((f.shape[0], 2*image.shape[1], 2*image.shape[2]))
    for z in range(f.shape[0]): # Number of filters
        for _h in range(image.shape[1]):      
            for _w in range(image.shape[2]):
                res[z, _h*s : _h*s + f_size, _w*s : _w*s + f_size] += np.sum((image[:, _h, _w].reshape(-1,1,1)) * f[z], axis=0)
        res[z,:,:] += b[z]*np.ones((image.shape[1]*2,image.shape[2]*2))
    return res # we will need new_in so we wont calc it again for the backprop

In [2]:
x = np.ones((4,4,4))

for c in range(4):
    for i in range(4):
        for j in range(4):
            x[c,i,j] = (i+1)*(j*2+1)*1.3 + c
print(x)
#k = np.random.rand(1,2,3,3)
k=np.ones((2,4,2,2))

counter = 1
for n in range(2):
    for c in range(4):
        for i in range(2):
            for j in range(2):
                k[n,c,i,j] = counter
                counter=counter+1

print(k)


b=np.zeros((2))

[[[ 1.3  3.9  6.5  9.1]
  [ 2.6  7.8 13.  18.2]
  [ 3.9 11.7 19.5 27.3]
  [ 5.2 15.6 26.  36.4]]

 [[ 2.3  4.9  7.5 10.1]
  [ 3.6  8.8 14.  19.2]
  [ 4.9 12.7 20.5 28.3]
  [ 6.2 16.6 27.  37.4]]

 [[ 3.3  5.9  8.5 11.1]
  [ 4.6  9.8 15.  20.2]
  [ 5.9 13.7 21.5 29.3]
  [ 7.2 17.6 28.  38.4]]

 [[ 4.3  6.9  9.5 12.1]
  [ 5.6 10.8 16.  21.2]
  [ 6.9 14.7 22.5 30.3]
  [ 8.2 18.6 29.  39.4]]]
[[[[ 1.  2.]
   [ 3.  4.]]

  [[ 5.  6.]
   [ 7.  8.]]

  [[ 9. 10.]
   [11. 12.]]

  [[13. 14.]
   [15. 16.]]]


 [[[17. 18.]
   [19. 20.]]

  [[21. 22.]
   [23. 24.]]

  [[25. 26.]
   [27. 28.]]

  [[29. 30.]
   [31. 32.]]]]


In [3]:
params = [k,b]
#res_ref,_ = convTransp(x,params)#padding = 0
res = convTransp_alg(x,params)
#print(res_ref)
#res = conv(x,params,1,1)
print(res)

[[[  98.4  109.6  171.2  192.8  244.   276.   316.8  359.2]
  [ 120.8  132.   214.4  236.   308.   340.   401.6  444. ]
  [ 134.8  151.2  280.4  317.6  426.   484.   571.6  650.4]
  [ 167.6  184.   354.8  392.   542.   600.   729.2  808. ]
  [ 171.2  192.8  389.6  442.4  608.   692.   826.4  941.6]
  [ 214.4  236.   495.2  548.   776.   860.  1056.8 1172. ]
  [ 207.6  234.4  498.8  567.2  790.   900.  1081.2 1232.8]
  [ 261.2  288.   635.6  704.  1010.  1120.  1384.4 1536. ]]

 [[ 277.6  288.8  516.8  538.4  756.   788.   995.2 1037.6]
  [ 300.   311.2  560.   581.6  820.   852.  1080.  1122.4]
  [ 397.2  413.6  875.6  912.8 1354.  1412.  1832.4 1911.2]
  [ 430.   446.4  950.   987.2 1470.  1528.  1990.  2068.8]
  [ 516.8  538.4 1234.4 1287.2 1952.  2036.  2669.6 2784.8]
  [ 560.   581.6 1340.  1392.8 2120.  2204.  2900.  3015.2]
  [ 636.4  663.2 1593.2 1661.6 2550.  2660.  3506.8 3658.4]
  [ 690.   716.8 1730.  1798.4 2770.  2880.  3810.  3961.6]]]


In [53]:
def maxpool(image, f=2 , s=2):
    
    h_range = int((image.shape[1] - f) / s) + 1     # (W - F + 2P) / S  
    w_range = int((image.shape[2] - f) / s) + 1     # (W - F + 2P) / S 
    out = np.zeros((image.shape[0], h_range, w_range))
    
    for z in range(image.shape[0]): # Number of channels
        for _h in range(h_range):      
            for _w in range(w_range):
                out[z, _h, _w] = np.max(image[z, _h*s : _h*s + f, _w*s : _w*s + f])
    return out

maxpool(x)

array([[[  7.8,  18.2,  28.6,  39. ],
        [ 15.6,  36.4,  57.2,  78. ],
        [ 23.4,  54.6,  85.8, 117. ],
        [ 31.2,  72.8, 114.4, 156. ]],

       [[  8.8,  19.2,  29.6,  40. ],
        [ 16.6,  37.4,  58.2,  79. ],
        [ 24.4,  55.6,  86.8, 118. ],
        [ 32.2,  73.8, 115.4, 157. ]],

       [[  9.8,  20.2,  30.6,  41. ],
        [ 17.6,  38.4,  59.2,  80. ],
        [ 25.4,  56.6,  87.8, 119. ],
        [ 33.2,  74.8, 116.4, 158. ]],

       [[ 10.8,  21.2,  31.6,  42. ],
        [ 18.6,  39.4,  60.2,  81. ],
        [ 26.4,  57.6,  88.8, 120. ],
        [ 34.2,  75.8, 117.4, 159. ]]])

In [54]:
params = [k,b]
res = conv(x,params,1,1)
print(res)

[[[  1998.    3930.6   5849.4   7768.2   9687.   11605.8  13524.6
     9399.6]
  [  3481.2   7004.4  10654.8  14305.2  17955.6  21606.   25256.4
    17547.6]
  [  4682.4   9664.2  15046.2  20428.2  25810.2  31192.2  36574.2
    25425.6]
  [  5883.6  12324.   19437.6  26551.2  33664.8  40778.4  47892.
    33303.6]
  [  7084.8  14983.8  23829.   32674.2  41519.4  50364.6  59209.8
    41181.6]
  [  8286.   17643.6  28220.4  38797.2  49374.   59950.8  70527.6
    49059.6]
  [  9487.2  20303.4  32611.8  44920.2  57228.6  69537.   81845.4
    56937.6]
  [  6169.2  13198.2  21201.   29203.8  37206.6  45209.4  53212.2
    36908.4]]

 [[  5108.4  10281.   15569.4  20857.8  26146.2  31434.6  36723.
    25988.4]
  [  9270.   19057.2  29446.8  39836.4  50226.   60615.6  71005.2
    50293.2]
  [ 12717.6  26771.4  42262.2  57753.   73243.8  88734.6 104225.4
    73896. ]
  [ 16165.2  34485.6  55077.6  75669.6  96261.6 116853.6 137445.6
    97498.8]
  [ 19612.8  42199.8  67893.   93586.2 119279.4 1449

In [58]:
k1= np.rot90(k,2,(2,3))
params = [k1, b]
res1 = conv_alg(x,params,1,0)
print(res1)

(2, 10, 10)
[[[  1998.    3930.6   5849.4   7768.2   9687.   11605.8  13524.6
     9399.6]
  [  3481.2   7004.4  10654.8  14305.2  17955.6  21606.   25256.4
    17547.6]
  [  4682.4   9664.2  15046.2  20428.2  25810.2  31192.2  36574.2
    25425.6]
  [  5883.6  12324.   19437.6  26551.2  33664.8  40778.4  47892.
    33303.6]
  [  7084.8  14983.8  23829.   32674.2  41519.4  50364.6  59209.8
    41181.6]
  [  8286.   17643.6  28220.4  38797.2  49374.   59950.8  70527.6
    49059.6]
  [  9487.2  20303.4  32611.8  44920.2  57228.6  69537.   81845.4
    56937.6]
  [  6169.2  13198.2  21201.   29203.8  37206.6  45209.4  53212.2
    36908.4]]

 [[  5108.4  10281.   15569.4  20857.8  26146.2  31434.6  36723.
    25988.4]
  [  9270.   19057.2  29446.8  39836.4  50226.   60615.6  71005.2
    50293.2]
  [ 12717.6  26771.4  42262.2  57753.   73243.8  88734.6 104225.4
    73896. ]
  [ 16165.2  34485.6  55077.6  75669.6  96261.6 116853.6 137445.6
    97498.8]
  [ 19612.8  42199.8  67893.   93586.2 1

In [59]:
print(res-res1)

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.09494702e-13
    1.81898940e-12  1.81898940e-12 -1.81898940e-12 -1.81898940e-12]
  [ 4.54747351e-13  0.00000000e+00  0.00000000e+00  1.81898940e-12
    0.00000000e+00  3.63797881e-12  0.00000000e+00 -3.63797881e-12]
  [ 9.09494702e-13  0.00000000e+00 -1.81898940e-12  3.63797881e-12
    0.00000000e+00  3.63797881e-12  0.00000000e+00  3.63797881e-12]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.63797881e-12
    0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.27595761e-12]
  [ 9.09494702e-13  1.81898940e-12  0.00000000e+00  3.63797881e-12
    0.00000000e+00  0.00000000e+00 -7.27595761e-12  0.00000000e+00]
  [ 0.00000000e+00 -3.63797881e-12  0.00000000e+00  7.27595761e-12
   -7.27595761e-12  0.00000000e+00  0.00000000e+00 -7.27595761e-12]
  [ 0.00000000e+00  0.00000000e+00 -7.27595761e-12 -7.27595761e-12
    0.00000000e+00 -1.45519152e-11  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -1.81898940e-12  0.00000000e+00  0.0

In [33]:
print(k)

[[[[ 1.  2.  3.]
   [ 4.  5.  6.]
   [ 7.  8.  9.]]

  [[10. 11. 12.]
   [13. 14. 15.]
   [16. 17. 18.]]

  [[19. 20. 21.]
   [22. 23. 24.]
   [25. 26. 27.]]

  [[28. 29. 30.]
   [31. 32. 33.]
   [34. 35. 36.]]]


 [[[37. 38. 39.]
   [40. 41. 42.]
   [43. 44. 45.]]

  [[46. 47. 48.]
   [49. 50. 51.]
   [52. 53. 54.]]

  [[55. 56. 57.]
   [58. 59. 60.]
   [61. 62. 63.]]

  [[64. 65. 66.]
   [67. 68. 69.]
   [70. 71. 72.]]]]


In [34]:
print(np.rot90(k,2,(2,3)))

[[[[ 9.  8.  7.]
   [ 6.  5.  4.]
   [ 3.  2.  1.]]

  [[18. 17. 16.]
   [15. 14. 13.]
   [12. 11. 10.]]

  [[27. 26. 25.]
   [24. 23. 22.]
   [21. 20. 19.]]

  [[36. 35. 34.]
   [33. 32. 31.]
   [30. 29. 28.]]]


 [[[45. 44. 43.]
   [42. 41. 40.]
   [39. 38. 37.]]

  [[54. 53. 52.]
   [51. 50. 49.]
   [48. 47. 46.]]

  [[63. 62. 61.]
   [60. 59. 58.]
   [57. 56. 55.]]

  [[72. 71. 70.]
   [69. 68. 67.]
   [66. 65. 64.]]]]
